In [1]:
import pandas as pd
import numpy as np
import requests
import json

from datetime import datetime

pd.options.display.float_format = '{:,.8f}'.format

In [2]:
req = requests.get('https://economia.awesomeapi.com.br/all/USD-BRL')
dolar = req.json()

In [3]:
float(dolar['USD']['bid'])

5.2265

In [4]:
api_key = 'bO8vmuXx8IJXH3QwwwazOigrxVmkIv1XykU49UJN5lcxIknWBd6ooEu2lOZ0tuTS'
api_secret = '4CZByMtKTGL7KdTfydBp2tCoGn9ZY4EYAtdP2DJRoaSE5mdXR2QXppJsj1kwCrCx'

In [5]:
from binance.spot import Spot as Client

client = Client()
client = Client(key=api_key, secret=api_secret)

spot_client = Client(api_key, api_secret, show_header=False)

In [7]:
# no LD - spot account amount OR earn results
# with LD - amount in earn flex
df_spot = pd.DataFrame(client.account()['balances'])[['asset', 'free']]
df_spot['free'] = df_spot['free'].astype(float)
df_spot = df_spot[df_spot['free']>0].sort_values('free', ascending=False)

df_spot.loc[df_spot['asset'].str.contains('LD'), 'asset'] = df_spot.loc[df_spot['asset'].str.contains('LD')]['asset'].str[2:]
df_spot = df_spot.groupby('asset').sum().sort_values('free', ascending=False)

In [8]:
df_spot

,free
asset,
MANA,81.96177674
ADA,2.14886505
BRL,1.28637339
ETH,0.03234614
BTC,0.00452079


In [16]:
# all staking
params = {"product": "STAKING", "txnType": "SUBSCRIPTION"}
staking = spot_client.staking_history(**params)

staking

[]

In [21]:
df_aux = pd.DataFrame(staking)
try:
    df_aux['amount'] = df_aux['amount'].astype(float)
    df_aux = df_aux[['amount', 'asset']].groupby('asset').sum()
    df_assets = df_spot.join(df_aux).fillna(0)
    df_assets = df_assets['free'] + df_assets['amount']
    df_assets = df_assets.to_frame('amount')
    df_assets
except:
    df_assets = df_spot.copy()
    df_assets.columns = ['amount']
    print('no staking')

no staking


In [23]:
ticker = spot_client.ticker_price()

In [24]:
df_ticker = pd.DataFrame(ticker)

In [25]:
cryptos = df_assets.index.tolist()
fiat_coin = ['BRL']

for i in fiat_coin:
    if i in cryptos:
        cryptos.remove(i)

In [26]:
df_curr_ticker = pd.DataFrame()
for i in cryptos:
    df_curr_ticker = df_curr_ticker.append(df_ticker.loc[df_ticker['symbol'].str.contains(i+'USDT')])

In [27]:
df_curr_ticker['symbol'] = df_curr_ticker['symbol'].str[:-4]
df_curr_ticker.set_index('symbol', inplace=True)
df_curr_ticker['price'] = df_curr_ticker['price'].astype(float)

In [28]:
df_assets = df_assets.join(df_curr_ticker).fillna(df_assets['amount'][1] / float(dolar['USD']['bid']))

In [29]:
df_assets['total_usd'] = df_assets['amount'] * df_assets['price']

In [30]:
df_assets['total_usd'].sum()

220.05041069055335

In [31]:
df_assets

,amount,price,total_usd
asset,,,
MANA,81.96065390,1.00350000,82.24751619
ADA,2.09516886,0.51030000,1.06916467
BRL,1.28633815,0.39971171,0.51416442
ETH,0.03234402,"1,225.38000000",39.63371523
BTC,0.00452018,"21,367.70000000",96.58585019


In [19]:
df_assets.to_csv('data/my_assets.csv', index=True)

In [20]:
# spot_client.savings_account()

In [21]:
# pd.DataFrame(spot_client.account_snapshot("SPOT", limit=30)['snapshotVos'])

In [20]:
# crypto + currency
spot_client.my_trades('BTCBRL')

[{'symbol': 'BTCBRL',
  'id': 14991229,
  'orderId': 298414317,
  'orderListId': -1,
  'price': '154620.00000000',
  'qty': '0.01325000',
  'quoteQty': '2048.71500000',
  'commission': '0.00001325',
  'commissionAsset': 'BTC',
  'time': 1652763795071,
  'isBuyer': True,
  'isMaker': False,
  'isBestMatch': True}]

In [ ]:
spot_client.my_trades('ADABTC')

In [ ]:
spot_client.my_trades('MANABTC')

In [ ]:
spot_client.my_trades('ETHBTC')

In [ ]:
spot_client.fiat_order_history(0)  # 0-buy | 1-sell

In [ ]:
spot_client.bnb_convertible_assets()

In [ ]:
start = int(datetime(2022, 5, 16, 23, 59, 59).timestamp() * 1000)
end = int(datetime(2022, 5, 19, 23, 59, 59).timestamp() * 1000)
# end = spot_client.time()['serverTime']

spot_client.convert_trade_history(startTime=start, endTime=end)

In [ ]:
spot_client.ticker_price('BTCUSDT')